In [1]:
# libraries
from pathlib import Path
import re
import markdown
from bs4 import BeautifulSoup


# directories
root = Path(".")

input = root / "input.md"
index = Path("index.html")
math_page = root / "pages/math.html"
code_page = root / "pages/code.html"
CV_page = root / "pages/CV.html"
math_cards_archive = root / "resources/math_cards_archive.md"
skeleton = root / "templates/skeleton.html"
README = Path("README.md")

# elements
button = """
<button class="w3-button w3-block w3-grey {classes}">
    {content}
</button>
"""

card = """
<div class="w3-white w3-text-dark-grey w3-card-4 w3-margin-bottom card">
    <div class="w3-container {classes}">
        {content}
    </div>
</div>
"""

figure = """
<figure class="w3-text-grey w3-center">
{img_tag}`
<figcaption>{caption}</figcaption>
</figure>
"""

# patterns
inline_math_pattern = re.compile(
    r"\\\(.*?\\\)",
    re.DOTALL,
)
block_math_pattern = re.compile(
    r"\\\[.*?\\\]",
    re.DOTALL,
)
code_block_pattern = re.compile(
    r"```python\s*(.*?)\s*```",
    re.DOTALL,
)
solution_pattern = re.compile(
    r"<!-- begin solution -->\s*(.*?)\s*<!-- end solution -->", re.DOTALL
)
figure_pattern = re.compile(r"<!-- figure:\s*(.*?)\s*-->\s*(<img.*?>)", re.DOTALL)

# functions
def soupify(file_name):
    with open(file_name, "r") as file:
        content = file.read()
    return BeautifulSoup(content, "lxml")


def stringify(file_name):
    with open(file_name, "r") as file:
        string = file.read()
    return string


def quarantine(string):
    inline_maths = inline_math_pattern.findall(string)
    block_maths = block_math_pattern.findall(string)
    code_blocks = code_block_pattern.findall(string)

    string = re.sub(
        inline_math_pattern,
        "INLINE-MATH-PLACEHOLDER",
        string,
    )
    string = re.sub(
        block_math_pattern,
        "BLOCK-MATH-PLACEHOLDER",
        string,
    )
    string = re.sub(
        code_block_pattern,
        "CODE-BLOCK-PLACEHOLDER",
        string,
    )

    return string, inline_maths, block_maths, code_blocks


def parse_quarantined(string):
    string = solution_pattern.sub(
        r'<div class="solution w3-margin-bottom"><b class="w3-text-green">Solution. </b>\1 &#x25A2;</div>',
        string,
    )
    string = figure_pattern.sub(
        r'<figure class="w3-text-grey w3-center">\2<figcaption>\1</figcaption></figure>',
        string,
    )

    return string


def parse_markdown(string):
    string, inline_maths, block_maths, code_blocks = quarantine(string)
    string = parse_quarantined(string)

    html = markdown.markdown(string)

    # return sensitive parts
    html = re.sub(
        "INLINE-MATH-PLACEHOLDER",
        lambda x: inline_maths.pop(0),
        html,
    )
    html = re.sub(
        "BLOCK-MATH-PLACEHOLDER",
        lambda x: block_maths.pop(0),
        html,
    )
    html = re.sub(
        "CODE-BLOCK-PLACEHOLDER",
        lambda x: f'<pre><code class="language-py">{code_blocks.pop(0)}</code></pre>',
        html,
    )

    return html


def create_card(string, classes=""):
    html = parse_markdown(string)
    html = card.format(content=html, classes=classes)
    soup = BeautifulSoup(html, "lxml").div
    if soup.select_one(".solution"):
        toggle_solution_button = button.format(
            content="Show/Hide Solution", classes="toggle-solution"
        )
        toggle_solution_button = BeautifulSoup(toggle_solution_button, "lxml").button
        soup.append(toggle_solution_button)

    return soup


def apply_skeleton(page, skeleton):
    open(page, "w").close()
    with open(skeleton, "r") as skeleton_file, open(page, "w") as page_file:
        content = skeleton_file.read()
        page_file.write(content)


def append_card(page, source, archive=None):
    with open(page, "r") as file:
        page_content = file.read()
    soup = BeautifulSoup(page_content.strip(), "lxml")
    if not isinstance(source, str):
        string = stringify(source)
    else:
        string = source
    card = create_card(string.strip())
    soup.select_one("#cards-container").append(card)
    with open(page, "w") as file:
        file.write(str(soup))
    if not archive is None:
        with open(archive, "a") as file:
            file.write(string.strip() + "\n\n<!-- separator -->\n\n")
    return True


def update_cards_from_archive(page, skeleton, archive):
  apply_skeleton(page, skeleton)
  with open(archive, "r") as file:
    strings = file.read()
  for string in strings.split(r"<!-- separator -->")[:-1]:
    append_card(page, string, archive=None)

  return True

In [2]:
update_cards_from_archive(index, skeleton, README)

True